# Notebook to fetch the input data for GW150914 from GWOSC and Zenodo

In [1]:
import h5ify
import numpy as np
from gwpy.timeseries import TimeSeries
import os

/mnt/home/smiller/src/time-domain-gw-inference/.venv/lib/python3.11/site-packages/gwpy/time/__init__.py:36: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  from lal import LIGOTimeGPS


### Get data

Download the `pesummary` result file from Zenodo:

In [2]:
!wget https://zenodo.org/record/6513631/files/IGWN-GWTC2p1-v2-GW150914_095045_PEDataRelease_mixed_cosmo.h5

--2025-07-11 18:50:33--  https://zenodo.org/record/6513631/files/IGWN-GWTC2p1-v2-GW150914_095045_PEDataRelease_mixed_cosmo.h5
Resolving zenodo.org (zenodo.org)... 188.185.43.25, 188.185.45.92, 188.185.48.194
Connecting to zenodo.org (zenodo.org)|188.185.43.25|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/6513631/files/IGWN-GWTC2p1-v2-GW150914_095045_PEDataRelease_mixed_cosmo.h5 [following]
--2025-07-11 18:50:34--  https://zenodo.org/records/6513631/files/IGWN-GWTC2p1-v2-GW150914_095045_PEDataRelease_mixed_cosmo.h5
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 134283070 (128M) [application/octet-stream]
Saving to: ‘IGWN-GWTC2p1-v2-GW150914_095045_PEDataRelease_mixed_cosmo.h5’

IGWN-GWTC2p1-v2-GW1 100%[===================>] 128.06M  5.64MB/s    in 24s     

2025-07-11 18:50:59 (5.23 MB/s) - ‘IGWN-GWTC2p1-v2-GW150914_095045_PEDataRelease_mixed_cosmo.h5’ saved [134283070/134283070]

Download the strain `.gwf` files from GWOSC (32 second, 16kHz):

In [3]:
!wget https://gwosc.org/eventapi/html/GWTC-1-confident/GW150914/v3/H-H1_GWOSC_16KHZ_R1-1126259447-32.gwf

--2025-07-11 18:50:59--  https://gwosc.org/eventapi/html/GWTC-1-confident/GW150914/v3/H-H1_GWOSC_16KHZ_R1-1126259447-32.gwf
Resolving gwosc.org (gwosc.org)... 131.215.113.72
Connecting to gwosc.org (gwosc.org)|131.215.113.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4050875 (3.9M) [application/octet-stream]
Saving to: ‘H-H1_GWOSC_16KHZ_R1-1126259447-32.gwf’

H-H1_GWOSC_16KHZ_R1 100%[===================>]   3.86M  4.79MB/s    in 0.8s    

2025-07-11 18:51:00 (4.79 MB/s) - ‘H-H1_GWOSC_16KHZ_R1-1126259447-32.gwf’ saved [4050875/4050875]



In [4]:
!wget https://gwosc.org/eventapi/html/GWTC-1-confident/GW150914/v3/L-L1_GWOSC_16KHZ_R1-1126259447-32.gwf

--2025-07-11 18:51:00--  https://gwosc.org/eventapi/html/GWTC-1-confident/GW150914/v3/L-L1_GWOSC_16KHZ_R1-1126259447-32.gwf
Resolving gwosc.org (gwosc.org)... 131.215.113.72
Connecting to gwosc.org (gwosc.org)|131.215.113.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3900380 (3.7M) [application/octet-stream]
Saving to: ‘L-L1_GWOSC_16KHZ_R1-1126259447-32.gwf’

L-L1_GWOSC_16KHZ_R1 100%[===================>]   3.72M  4.27MB/s    in 0.9s    

2025-07-11 18:51:02 (4.27 MB/s) - ‘L-L1_GWOSC_16KHZ_R1-1126259447-32.gwf’ saved [3900380/3900380]



### Load in the data

First, load in the posterior file. This also contains metadata for the corresponding run.

In [5]:
# Posterior
posterior = h5ify.load('IGWN-GWTC2p1-v2-GW150914_095045_PEDataRelease_mixed_cosmo.h5')

From the posterior file, get the PSDs:

In [6]:
# PSDs
H1_psd = posterior['C01:IMRPhenomXPHM']['psds']['H1']
L1_psd = posterior['C01:IMRPhenomXPHM']['psds']['L1']

# Save them as text files to be loaded into the TD inference code later
np.savetxt('H1_psd.txt', H1_psd, delimiter=' ')
np.savetxt('L1_psd.txt', L1_psd, delimiter=' ')

Next, load in the strain data:

In [7]:
# get trigger time
t0 = float(posterior['C01:IMRPhenomXPHM']['config_file']['config']['trigger-time'][0].decode('utf-8'))
print('trigger time', t0)

# get segment duratino
duration = float(posterior['C01:IMRPhenomXPHM']['config_file']['config']['duration'][0].decode('utf-8'))
print('duration', duration)

# cycle through IFOs
for ifo in ['H1', 'L1']:

    # get gwf data
    gwf_data = TimeSeries.read(f'{ifo[0]}-{ifo}_GWOSC_16KHZ_R1-1126259447-32.gwf', f'{ifo}:GWOSC-16KHZ_R1_STRAIN')
    
    # convert to a dictionary with +/- half the duration on each side of the trigger time
    dt = duration / 2
    mask = (gwf_data.times.value > t0 - dt) & (gwf_data.times.value <  t0 + dt)
    data = {'strain':gwf_data.value[mask], 'times':gwf_data.times.value[mask]}

    print('start time, end time:', data['times'][0], data['times'][-1])
    
    # save as .h5 file
    if not os.path.exists(f'{ifo}_strain.h5'):
        h5ify.save(f'{ifo}_strain.h5', data)

trigger time 1126259462.391
duration 4.0
start time, end time: 1126259460.3910522 1126259464.3909912
start time, end time: 1126259460.3910522 1126259464.3909912


Other relevant quantities for analysis: 

In [8]:
quantities = ['minimum-frequency', 'maximum-frequency', 'reference-frequency', 'sampling-frequency']

for q in quantities: 
    print(q, posterior['C01:IMRPhenomXPHM']['config_file']['config'][q][0].decode('utf-8'))

minimum-frequency { 'H1': 20, 'L1': 20,  }
maximum-frequency { 'H1': 896, 'L1': 896,  }
reference-frequency 20
sampling-frequency 2048
